# Data Merging Basics

Outline
- [1 Inner Join](#inner-join)
- [&nbsp;&nbsp; 1.1 What column to merge on?](#clm-merge-on)
- [&nbsp;&nbsp; 1.2 Your first inner join](#chg-sales)
- [&nbsp;&nbsp; 1.3 Inner joins and number of rows returned](#inr-jn-num-rows)
- [2 One-to-many relationships](#one-to-many)
- [&nbsp;&nbsp; 2.1 One-to-many classification](#otm-class)
- [&nbsp;&nbsp; 2.2 One-to-many merge](#otm-merge)
- [3 Merging multiple DataFrames](#merge-multi-df)
- [&nbsp;&nbsp; 3.1 Total riders in a month](#total-riders)
- [&nbsp;&nbsp; 3.2 Three table merge](#three-tbl-merge)
- [&nbsp;&nbsp; 3.3 One-to-many merge with multiple tables](#oto-merge-multi-tbls)

<a id="inner-join"></a>
# 1 Inner Join
<!-- %%HTML -->
<div align="middle">
<video width="60%" controls>
      <source src="./../../res/videos/1.data-merging-basics/1.inner_join.mp4" type="video/mp4">
</video></div>



<a id="clm-merge-on"></a>
## 1.1 What column to merge on?
Chicago provides a list of taxicab owners and vehicles licensed to operate within the city, for public safety. Your goal is to merge two tables together. One table is called taxi_owners, with info about the taxi cab company owners, and one is called taxi_veh, with info about each taxi cab vehicle. Both the taxi_owners and taxi_veh tables have been loaded for you and you can explore them in the IPython shell.

Choose the column you would use to merge the two tables on using the .merge() method.

In [31]:
import pandas as pd

taxi_owners = pd.read_pickle("../../data/1/taxi_owners.p")
taxi_veh = pd.read_pickle("../../data/1/taxi_vehicles.p")

In [32]:
taxi_owners.head(4)

,rid,vid,owner,address,zip
0,T6285,6285,AGEAN TAXI LLC,4536 N. ELSTON AVE.,60630
1,T4862,4862,MANGIB CORP.,5717 N. WASHTENAW AVE.,60659
2,T1495,1495,"FUNRIDE, INC.",3351 W. ADDISON ST.,60618
3,T4231,4231,ALQUSH CORP.,6611 N. CAMPBELL AVE.,60645


In [33]:
taxi_veh.head(4)

,vid,make,model,year,fuel_type,owner
0,2767,TOYOTA,CAMRY,2013,HYBRID,SEYED M. BADRI
1,1411,TOYOTA,RAV4,2017,HYBRID,DESZY CORP.
2,6500,NISSAN,SENTRA,2019,GASOLINE,AGAPH CAB CORP
3,2746,TOYOTA,CAMRY,2013,HYBRID,"MIDWEST CAB CO, INC"


Answer => we would pick `vid` column to merge both tables

<a id="chg-sales"></a>
## 1.2 Your first inner join
You have been tasked with figuring out what the most popular types of fuel used in Chicago taxis are. To complete the analysis, you need to merge the taxi_owners and taxi_veh tables together on the vid column. You can then use the merged table along with the .value_counts() method to find the most common fuel_type.

Since you'll be working with pandas throughout the course, the package will be preloaded for you as pd in each exercise in this course. Also the taxi_owners and taxi_veh DataFrames are loaded for you.

Merge taxi_owners with taxi_veh on the column vid, and save the result to taxi_own_veh.

In [34]:
taxi_own_veh = pd.merge(taxi_owners, taxi_veh, on="vid")

Set the left and right table suffixes for overlapping columns of the merge to _own and _veh, respectively.

In [35]:
taxi_own_veh = pd.merge(taxi_owners, taxi_veh, on="vid", suffixes=("_own", "_veh"))
taxi_own_veh

,rid,vid,owner_own,address,zip,make,model,year,fuel_type,owner_veh
0,T6285,6285,AGEAN TAXI LLC,4536 N. ELSTON AVE.,60630,NISSAN,ALTIMA,2011,HYBRID,AGEAN TAXI LLC
1,T4862,4862,MANGIB CORP.,5717 N. WASHTENAW AVE.,60659,HONDA,CRV,2014,GASOLINE,MANGIB CORP.
2,T1495,1495,"FUNRIDE, INC.",3351 W. ADDISON ST.,60618,TOYOTA,SIENNA,2015,GASOLINE,"FUNRIDE, INC."
3,T4231,4231,ALQUSH CORP.,6611 N. CAMPBELL AVE.,60645,TOYOTA,CAMRY,2014,HYBRID,ALQUSH CORP.
4,T5971,5971,EUNIFFORD INC.,3351 W. ADDISON ST.,60618,TOYOTA,SIENNA,2015,GASOLINE,EUNIFFORD INC.
...,...,...,...,...,...,...,...,...,...,...
3514,T4453,4453,IMAGIN CAB CORP,3351 W. ADDISON ST.,60618,FORD,ESCAPE,2010,HYBRID,IMAGIN CAB CORP
3515,T121,121,TRIBECA CAB CORP,4536 N. ELSTON AVE.,60630,FORD,C-MAX,2014,HYBRID,TRIBECA CAB CORP
3516,T3465,3465,AMIR EXPRESS INC,3351 W. ADDISON ST.,60618,TOYOTA,CAMRY,2014,HYBRID,AMIR EXPRESS INC
3517,T1962,1962,KARY CAB COMPANY,4707 N. KENTON AVE.,60630,TOYOTA,CAMRY,2014,HYBRID,KARY CAB COMPANY


Select the fuel_type column from taxi_own_veh and print the value_counts() to find the most popular fuel_types used.

In [36]:
taxi_own_veh['fuel_type'].value_counts()

fuel_type
HYBRID                    2792
GASOLINE                   611
FLEX FUEL                   89
COMPRESSED NATURAL GAS      27
Name: count, dtype: int64

<a id="inr-jn-num-rows"></a>
## 1.3 Inner joins and number of rows returned

All of the merges you have studied to this point are called inner joins. It is necessary to understand that inner joins only return the rows with matching values in both tables. You will explore this further by reviewing the merge between the wards and census tables, then comparing it to merges of copies of these tables that are slightly altered, named wards_altered, and census_altered. The first row of the wards column has been changed in the altered tables. You will examine how this affects the merge between them. The tables have been loaded for you.

For this exercise, it is important to know that the wards and census tables start with 50 rows.

In [37]:
wards = pd.read_pickle("../../data/1/ward.p")
census = pd.read_pickle("../../data/1/census.p")
wards_altered = wards.copy()
wards_altered.iat[0,0] = "61" # faster than iloc[0,0] at accessing and  modifying single cells
census_altered = census.copy()
census_altered.iat[0,0] = None

Merge wards and census on the ward column and save the result to wards_census.

In [38]:
wards_census = wards.merge(census, on="ward")

Merge the wards_altered and census tables on the ward column, and notice the difference in returned rows.

In [39]:
wards_altered['ward'].head()  # only difference is first ward is set to 61 instead of 1 in ward
wards_altered_census = wards_altered.merge(census, on="ward")
print('wards_altered_census table shape:', wards_altered_census.shape)

wards_altered_census table shape: (49, 9)



Merge the wards and census_altered tables on the ward column, and notice the difference in returned rows.

In [40]:
wards_census_altered = wards.merge(census_altered, on="ward")
wards_census_altered.shape

(49, 9)

<a id="one-to-many"></a>
# 2 One-to-many relationships
<a id="otm-class"></a>
## 2.1 One-to-many classification
Understanding the difference between a one-to-one and one-to-many relationship is a useful skill. In this exercise, consider a set of tables from an e-commerce website. The hypothetical tables are the following:

  - A customer table with information about each customer
  - A cust_tax_info table with customers unique tax IDs
  - An orders table with information about each order
  - A products table with details about each unique product sold
  - An inventory table with information on how much total inventory is available to sell for each product


One-to-one:
  - the relationship between products and inventory.
  - the relationship customer and cust_tax_info.

One-to-many:
  - the relationship between customers and orders.
  - the relationship between products and orders.

<a id="otm-merge"></a>
## 2.2 One-to-many merge
A business may have one or multiple owners. In this exercise, you will continue to gain experience with one-to-many merges by merging a table of business owners, called biz_owners, to the licenses table. Recall from the video lesson, with a one-to-many relationship, a row in the left table may be repeated if it is related to multiple rows in the right table. In this lesson, you will explore this further by finding out what is the most common business owner title. (i.e., secretary, CEO, or vice president)

The licenses and biz_owners DataFrames are loaded for you.

In [41]:
licenses = pd.read_pickle("../../data/1/licenses.p")
biz_owners = pd.read_pickle("../../data/1/business_owners.p")

display(licenses.head(4))
display(biz_owners.head(4))

,account,ward,aid,business,address,zip
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633
2,10002,14,775,CELINA DELI,5089 S ARCHER AVE,60632
3,10005,12,NaN,KRAFT FOODS NORTH AMERICA,2005 W 43RD ST,60609


,account,first_name,last_name,title
0,10,PEARL,SHERMAN,PRESIDENT
1,10,PEARL,SHERMAN,SECRETARY
2,10002,WALTER,MROZEK,PARTNER
3,10002,CELINA,BYRDAK,PARTNER


Starting with the licenses table on the left, merge it to the biz_owners table on the column account, and save the results to a variable named licenses_owners

In [42]:
licenses_owners = licenses.merge(biz_owners, on="account")

Group licenses_owners by title and count the number of accounts for each title. Save the result as counted_df

In [43]:
counted_df = licenses_owners.groupby("title").agg({"account" : "count"})
counted_df

,account
title,
ASST. SECRETARY,111
BENEFICIARY,4
CEO,110
DIRECTOR,146
EXECUTIVE DIRECTOR,10
GENERAL PARTNER,21
INDIVIDUAL,268
LIMITED PARTNER,26
MANAGER,134


Sort counted_df by the number of accounts in descending order, and save this as a variable named sorted_df.

In [44]:
sorted_df = counted_df.sort_values(by="account", ascending=False)

Use the .head() method to print the first few rows of the sorted_df.

In [45]:
sorted_df.head()

,account
title,
PRESIDENT,6259
SECRETARY,5205
SOLE PROPRIETOR,1658
OTHER,1200
VICE PRESIDENT,970



<a id="merge-multi-df"></a>
# 3 Merging multiple DataFrames
<a id="total-riders"></a>
## 3.1 Total riders in a month
Your goal is to find the total number of rides provided to passengers passing through the Wilson station (station_name == 'Wilson') when riding Chicago's public transportation system on weekdays (day_type == 'Weekday') in July (month == 7). Luckily, Chicago provides this detailed data, but it is in three different tables. You will work on merging these tables together to answer the question. This data is different from the business related data you have seen so far, but all the information you need to answer the question is provided.

The cal, ridership, and stations DataFrames have been loaded for you. The relationship between the tables can be seen in the diagram below.
<br>
<div style="text-align: center;">
    <img src="../../res/images/cta_L_diagram.png" width="800" height="400">
</div>

In [46]:
cal = pd.read_pickle("../../data/1/cta_calendar.p")
ridership = pd.read_pickle("../../data/1/cta_ridership.p")
stations = pd.read_pickle("../../data/1/stations.p")

Merge the ridership and cal tables together, starting with the ridership table on the left and save the result to the variable ridership_cal. If you code takes too long to run, your merge conditions might be incorrect.

In [47]:
ridership_cal = ridership.merge(cal, on=["year", "month", "day"])
ridership_cal

,station_id,year,month,day,rides,day_type
0,40010,2019,1,1,576,Sunday/Holiday
1,40080,2019,1,1,1839,Sunday/Holiday
2,40770,2019,1,1,2724,Sunday/Holiday
3,40120,2019,1,1,754,Sunday/Holiday
4,40540,2019,1,1,2175,Sunday/Holiday
...,...,...,...,...,...,...
3280,40540,2019,12,31,4355,Weekday
3281,41260,2019,12,31,1228,Weekday
3282,41500,2019,12,31,1685,Weekday
3283,41440,2019,12,31,1370,Weekday


Extend the previous merge to three tables by also merging the stations table.

In [48]:
ridership_cal_stations = ridership.merge(cal, on=["year", "month", "day"]) \
                                    .merge(stations, on="station_id")

Create a variable called filter_criteria to select the appropriate rows from the merged table so that you can sum the rides column.

In [49]:
# Create a filter to filter ridership_cal_stations
filter_criteria = (
    (ridership_cal_stations["month"] == 7) &
    (ridership_cal_stations["day_type"] == "Weekday") &
    (ridership_cal_stations["station_name"] == "Wilson")
)

# Use .loc and the filter to select for rides
ridership_cal_stations.loc[filter_criteria, "rides"].sum()

140005

<a id="three-tbl-merge"></a>
## 3.2 Three table merge
To solidify the concept of a three DataFrame merge, practice another exercise. A reasonable extension of our review of Chicago business data would include looking at demographics information about the neighborhoods where the businesses are. A table with the median income by zip code has been provided to you. You will merge the licenses and wards tables with this new income-by-zip-code table called zip_demo.

The licenses, wards, and zip_demo DataFrames have been loaded for you.

In [50]:
licenses = pd.read_pickle("../../data/1/licenses.p")
zip_demo = pd.read_csv("../../data/1/zip_demo.csv", index_col=0)
zip_demo['zip'] = zip_demo['zip'].astype(str) # to be able to merge it with licenses on zip column they must be of same type

Starting with the licenses table, merge to it the zip_demo table on the zip column. Then merge the resulting table to the wards table on the ward column. Save result of the three merged tables to a variable named licenses_zip_ward.

In [51]:
licenses_zip_ward = licenses.merge(zip_demo, on="zip") \
                            .merge(wards, on="ward")

Group the results of the three merged tables by the column alderman and find the median income.

In [52]:
licenses_zip_ward.groupby("alderman").agg({"income": "median"})

,income
alderman,
Ameya Pawar,66246.0
Anthony A. Beale,38206.0
Anthony V. Napolitano,82226.0
Ariel E. Reyboras,41307.0
Brendan Reilly,110215.0
Brian Hopkins,87143.0
Carlos Ramirez-Rosa,66246.0
Carrie M. Austin,38206.0
Chris Taliaferro,55566.0


<a id="oto-merge-multi-tbls"></a>
## 3.3 One-to-many merge with multiple tables
In this exercise, assume that you are looking to start a business in the city of Chicago. Your perfect idea is to start a company that uses goats to mow the lawn for other businesses. However, you have to choose a location in the city to put your goat farm. You need a location with a great deal of space and relatively few businesses and people around to avoid complaints about the smell. You will need to merge three tables to help you choose your location. The land_use table has info on the percentage of vacant land by city ward. The census table has population by ward, and the licenses table lists businesses by ward.

The land_use, census, and licenses tables have been loaded for you.

In [53]:
land_use = pd.read_pickle("../../data/1/land_use.p")
census = pd.read_pickle("../../data/1/census.p")
licenses = pd.read_pickle("../../data/1/licenses.p")

display(land_use.head())
display(census.head())
display(licenses.head())

,ward,residential,commercial,industrial,vacant,other
0,1,41,9,2,2,46
1,2,31,11,6,2,50
2,3,20,5,3,13,59
3,4,22,13,0,7,58
4,5,25,3,1,3,68


,ward,pop_2000,pop_2010,change,address,zip
0,1,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637


,account,ward,aid,business,address,zip
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633
2,10002,14,775,CELINA DELI,5089 S ARCHER AVE,60632
3,10005,12,NaN,KRAFT FOODS NORTH AMERICA,2005 W 43RD ST,60609
4,10044,44,638,NEYBOUR'S TAVERN & GRILLE,3651 N SOUTHPORT AVE,60613


Merge land_use and census on the ward column. Merge the result of this with licenses on the ward column, using the suffix _cen for the left table and _lic for the right table. Save this to the variable land_cen_lic.

In [54]:
land_cen_lic = land_use.merge(census, on="ward") \
                        .merge(licenses, on="ward", suffixes=("_cen", "_lic"))


land_cen_lic.head(3)

,ward,residential,commercial,industrial,vacant,other,pop_2000,pop_2010,change,address_cen,zip_cen,account,aid,business,address_lic,zip_lic
0,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,12024,NaN,DIGILOG ELECTRONICS,1038 N ASHLAND AVE,60622
1,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14446,743,EMPTY BOTTLE INC,1035 N WESTERN AVE 1ST,60622
2,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14624,775,LITTLE MEL'S HOT DOG,2205 N CALIFORNIA AVE,60647


Group land_cen_lic by ward, pop_2010 (the population in 2010), and vacant, then count the number of accounts. Save the results to pop_vac_lic.

In [55]:
pop_vac_lic = land_cen_lic.groupby(by=["ward", "pop_2010", "vacant"], as_index=False).agg({"account" : "count"})

Sort pop_vac_lic by vacant, account, andpop_2010 in descending, ascending, and ascending order respectively. Save it as sorted_pop_vac_lic.

In [56]:
sorted_pop_vac_lic = pop_vac_lic.sort_values(by=["vacant", "account", "pop_2010"], ascending=[False, True, True])
sorted_pop_vac_lic

,ward,pop_2010,vacant,account
47,7,51581,19,80
12,20,52372,15,123
1,10,51535,14,130
16,24,54909,13,98
7,16,51954,13,156
22,3,53039,13,173
20,28,55199,11,189
46,6,52341,8,149
27,34,51599,7,99
14,22,53515,7,156
